# In this Notebook  : 
We are going define classes to have a cleaner implementation and do more complex tasks with the plots.
We will have a usefull use of the interaction.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook

###  Class
We first define a **class** for the interaction to be cleaner. Doing so, we will be able to **store information**, and do backward actions on the ones which we've done before.

We just declare the class, **initialize** it (`__init__`) and define a `__call__` **function**. The **arguments** of the class are just a blank figure and an axe (in this notebook I use matplotlib **subplots** instead of just a figure, which allows more complex plots. (You just need to know that the figure is the canvas, and the axes are the plots inside the canvas.)

During the initialisation, we make the figure and the axes a property of the class (`self`), we plot the random plot we used before (see tutorial 1), and we create an empty list (`self.points`), which will store the points we add while clicking. We also link the interactive canvas to a press button action, as we used to. The only difference is that we don't link it to a specific function, but to `self`, which means the `__call__` function.

The call function will be called each time we click : it's the equivalent of the function we created in tutorial 1 defining what we wanted to do while interacting. 

Here, I kept the drawing color interaction. But now that we can store the added points, the left click (or `ctrl` click) will **remove the latest drown point** !

# Try it :

In [ ]:
class Interact:
    def __init__(self, fig, ax):
        self.ax = ax
        fig.canvas.mpl_connect('button_press_event', self)
        self.plot = plt.scatter(np.random.uniform(0, 10, 10), np.random.uniform(0, 10, 10))
        self.points = []

    def __call__(self, event):
        
            click_x = event.xdata
            click_y = event.ydata
            plt.title(f"I'm clicking on : (x, y) = ({click_x:.2f}, {click_y:.2f})")
            if (click_x > plt.xlim()[1]/2):
                color = 'red'
            else:
                color = 'blue'
            if event.button == 1:
                marker = 'o'
                self.points.append(self.ax.scatter(event.xdata, event.ydata, marker=marker, c=color))
            if event.button == 3:
                try:
                    self.points[-1].remove()
                    self.points.pop(len(self.points)-1)

                except IndexError:
                    plt.title("Ooops, you want to remove a point but \n there is no more !")


fig, ax = plt.subplots(figsize=(5, 5))
interact = Interact(fig, ax)


### Cool right ?

Ok, now that we have the basis and a clean way to work, let's have our first usefull interactive plot.

We are going to plot a 2D catalogue (let's play with galaxies, but you can use any table with more than 2 columns). The idea is to have a scatter plot of two of the parameters, and to print the values of an other parameter when clicking to the point of interest.

First, lets import the data : 

In [ ]:
cat = np.load('../data/Galaxy_catalogue.npy', allow_pickle=True).item()

The catalogue is now stored in `cat`. It's a simple dictionnary containing 300 galaxies. For each galaxy (rows), we have 4 parameters (columns) : the magnitude `mag`, the radius and the ellipticity. The magnitude is an indicator of the brightness of a galaxy.

To do what we want, we first need a function to recognize the data point we are clicking. Don't look to much at the function, it's not the point of the notebook. 
Just for you to know, the function returns the closest data point to a clicked position on a graph. We will not necessarly need to click exactly on the point, this function will do the job for us. In addition, it returns the index of the corresponding data point, so we can print the other parameters of the galaxy.

In [ ]:
from scipy.spatial import KDTree
def find_closest_point(coords, cat, p1, p2):
    searching_for = np.zeros((1, 2))
    searching_for[0, 0] = coords[0]
    searching_for[0, 1] = coords[1]

    searching_in = np.zeros(((len(cat[next(iter(cat))]), 2)))
    searching_in[:, 0] = cat[p1]
    searching_in[:, 1] = cat[p2]
    _, match_index = KDTree(searching_in).query(searching_for)
    closestx, closesty = searching_in[match_index][0]

    return match_index[0], closestx, closesty

# New Interaction !
Well let's do it ! The initialisation plots the radii of the galaxies wrt to their magnitude. Each time you'll click on the plot, the closest corresponding galaxy will be found, and its ellipticity will be printed !

# Try it :

In [ ]:
class Explore_catalogue:
    def __init__(self, fig, ax, cat, param1='mag', param2='radius', param3='ellipticity'):
        self.ax = ax
        self.param1 = param1
        self.param2 = param2
        self.param3 = param3
        self.cat = cat
        fig.canvas.mpl_connect('button_press_event', self)
        ax.scatter(cat[param1], cat[param2])

    def __call__(self, event):
        if event.button == 1:
            click_x = event.xdata
            click_y = event.ydata
            match_index, closest_x, closest_y = find_closest_point((click_x, click_y), self.cat, self.param1, self.param2)
            self.ax.set_title(f"clicked point : ({click_x:.2f}, {click_y:.2f}) \n "+
                              f"closest point : ({closest_x:.2f}, {closest_y:.2f}) \n"+
                             f"corresponding ellipticity:{cat[self.param3][match_index]:.2f}")
            
fig, ax = plt.subplots(figsize=(5, 5))
interact = Explore_catalogue(fig, ax, cat)


# Can be useful while exploring catalogs right ?
Well, of course you could have colorcode the ellipitcity, but you can always do that and use the interaction to print a 4th parameter !

Let's add a cool behavior : I want to plot the galaxy of the clicked point next to the scatter plot. Here I'll just use a overly simple toy model to create galaxies (galaxies ?? I doubt it... Don't look at the code please...) parameterized by the magnitude, the radius and ellipticity. What can be cool, with exactly the same code, is to show the images if you have the corresponfing galaxies to your catalogue !

Final thing : If you left click anywhere in the plot, it will create a new galaxy with the clicked mag and radius, and a random ellipticity.

# Try it :

In [ ]:
def gaussian_model(mag, rad, ell):
    rad = rad/2
    x, y = np.meshgrid(np.linspace(-32, 32, 64), np.linspace(-32, 32 ,64)) 
    dst = np.sqrt(x*x+y*y) 
    sigma = rad/2
    muu = 0
    
    galaxy = np.exp(-( (dst-muu)**2 / ( 2.0 * sigma**2 ) ) ) * 10**(-0.4*(mag-23.5))
    
    y, x = np.ogrid[-32:32, -32:32]
    mask = (x*x)/(rad) + (y*y)/(rad*(1-ell)) > rad
    
    galaxy[mask] = 0
    galaxy += np.random.normal(0, 0.1, (64, 64))
    return(galaxy)

In [ ]:
class Explore_catalogue:
    def __init__(
        self, 
        fig, 
        ax, 
        cat, 
        param1='mag', 
        param2='radius', 
        param3='ellipticity',
    ):
        
        self.plot_cat = ax[0]
        self.new_galaxies = []
        self.image = ax[1]
        self.param1 = param1
        self.param2 = param2
        self.param3 = param3
        self.cat = cat
        fig.canvas.mpl_connect('button_press_event', self)
        self.sc_plot = self.plot_cat.scatter(cat[param1], cat[param2], s=5, c="#1f77b4")
        self.size_data = len(cat[param1])
        
        # Reserve some space on top for the title
        fig.subplots_adjust(top=0.8)

    def __call__(self, event):
        click_x = event.xdata
        click_y = event.ydata
        
        if event.button == 1:
            match_index, closest_x, closest_y = find_closest_point(
                (click_x, click_y), 
                self.cat, 
                self.param1, 
                self.param2,
            )
            
            self.plot_cat.set_title(
                f"clicked point: ({click_x:.2f}, {click_y:.2f}) \n" +
                f"closest point: ({closest_x:.2f}, {closest_y:.2f}) \n" +
                f"corresponding ellipticity: {cat[self.param3][match_index]:.2f}",
            )
            
            # Give the nearest point the color red
            colors = ["#1f77b4"] * self.size_data
            colors[match_index] = "red"
            self.sc_plot.set_color(colors)
            
            self.image.imshow(gaussian_model(
                self.cat['mag'][match_index], 
                self.cat['radius'][match_index], 
                self.cat['ellipticity'][match_index],
            ))
        
        if event.button == 3:
            self.new_galaxies.append(self.plot_cat.scatter(
                event.xdata, 
                event.ydata, 
                marker='*', 
                c='red',
            ))
            
            self.plot_cat.set_title(f"new galaxy : ({click_x:.2f}, {click_y:.2f}) \n")
            self.image.imshow(gaussian_model(click_x, click_y, np.random.uniform(0,1)))

fig, ax = plt.subplots(1, 2, figsize=(7, 3.5))
interact = Explore_catalogue(fig, ax, cat)
